In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

In [2]:
tf.enable_eager_execution()

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model


In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3394, train_loss: 2.19944477, train_accuracy: 0.3594, test_Accuracy: 0.2045
Epoch: [ 0] [    1/  468] time: 0.4819, train_loss: 2.17719412, train_accuracy: 0.3750, test_Accuracy: 0.3452
Epoch: [ 0] [    2/  468] time: 0.6198, train_loss: 2.12102365, train_accuracy: 0.4375, test_Accuracy: 0.4523
Epoch: [ 0] [    3/  468] time: 0.7517, train_loss: 2.02887559, train_accuracy: 0.6172, test_Accuracy: 0.5662
Epoch: [ 0] [    4/  468] time: 0.8864, train_loss: 1.91317272, train_accuracy: 0.7500, test_Accuracy: 0.6493
Epoch: [ 0] [    5/  468] time: 1.0714, train_loss: 1.88816452, train_accuracy: 0.6953, test_Accuracy: 0.6807
Epoch: [ 0] [    6/  46

Epoch: [ 0] [   72/  468] time: 10.9343, train_loss: 0.30018926, train_accuracy: 0.9297, test_Accuracy: 0.9001
Epoch: [ 0] [   73/  468] time: 11.0745, train_loss: 0.48262343, train_accuracy: 0.8438, test_Accuracy: 0.9020
Epoch: [ 0] [   74/  468] time: 11.2258, train_loss: 0.25989312, train_accuracy: 0.9297, test_Accuracy: 0.9059
Epoch: [ 0] [   75/  468] time: 11.3815, train_loss: 0.34032080, train_accuracy: 0.9141, test_Accuracy: 0.9129
Epoch: [ 0] [   76/  468] time: 11.5279, train_loss: 0.31457752, train_accuracy: 0.9297, test_Accuracy: 0.9111
Epoch: [ 0] [   77/  468] time: 11.6719, train_loss: 0.21709931, train_accuracy: 0.9297, test_Accuracy: 0.9100
Epoch: [ 0] [   78/  468] time: 11.8062, train_loss: 0.20417790, train_accuracy: 0.9375, test_Accuracy: 0.9074
Epoch: [ 0] [   79/  468] time: 11.9715, train_loss: 0.25555706, train_accuracy: 0.9062, test_Accuracy: 0.9080
Epoch: [ 0] [   80/  468] time: 12.1115, train_loss: 0.30345428, train_accuracy: 0.9219, test_Accuracy: 0.9090
E

Epoch: [ 0] [  146/  468] time: 21.7325, train_loss: 0.28452241, train_accuracy: 0.8984, test_Accuracy: 0.9299
Epoch: [ 0] [  147/  468] time: 21.8905, train_loss: 0.14219598, train_accuracy: 0.9375, test_Accuracy: 0.9285
Epoch: [ 0] [  148/  468] time: 22.0327, train_loss: 0.34261569, train_accuracy: 0.8984, test_Accuracy: 0.9255
Epoch: [ 0] [  149/  468] time: 22.1642, train_loss: 0.18424171, train_accuracy: 0.9453, test_Accuracy: 0.9211
Epoch: [ 0] [  150/  468] time: 22.2994, train_loss: 0.21149266, train_accuracy: 0.9375, test_Accuracy: 0.9181
Epoch: [ 0] [  151/  468] time: 22.4346, train_loss: 0.23840015, train_accuracy: 0.9062, test_Accuracy: 0.9188
Epoch: [ 0] [  152/  468] time: 22.5775, train_loss: 0.15503831, train_accuracy: 0.9531, test_Accuracy: 0.9241
Epoch: [ 0] [  153/  468] time: 22.7181, train_loss: 0.18364502, train_accuracy: 0.9531, test_Accuracy: 0.9276
Epoch: [ 0] [  154/  468] time: 22.8521, train_loss: 0.14786641, train_accuracy: 0.9609, test_Accuracy: 0.9292
E

Epoch: [ 0] [  220/  468] time: 32.6111, train_loss: 0.18720147, train_accuracy: 0.9375, test_Accuracy: 0.9420
Epoch: [ 0] [  221/  468] time: 32.7469, train_loss: 0.15086666, train_accuracy: 0.9531, test_Accuracy: 0.9428
Epoch: [ 0] [  222/  468] time: 32.8805, train_loss: 0.11535162, train_accuracy: 0.9766, test_Accuracy: 0.9422
Epoch: [ 0] [  223/  468] time: 33.0208, train_loss: 0.22483592, train_accuracy: 0.9375, test_Accuracy: 0.9409
Epoch: [ 0] [  224/  468] time: 33.1541, train_loss: 0.18944004, train_accuracy: 0.9375, test_Accuracy: 0.9385
Epoch: [ 0] [  225/  468] time: 33.2952, train_loss: 0.16352424, train_accuracy: 0.9453, test_Accuracy: 0.9372
Epoch: [ 0] [  226/  468] time: 33.5025, train_loss: 0.26706791, train_accuracy: 0.9219, test_Accuracy: 0.9369
Epoch: [ 0] [  227/  468] time: 33.6388, train_loss: 0.27346334, train_accuracy: 0.9219, test_Accuracy: 0.9366
Epoch: [ 0] [  228/  468] time: 33.7885, train_loss: 0.13112928, train_accuracy: 0.9688, test_Accuracy: 0.9380
E

Epoch: [ 0] [  294/  468] time: 43.6454, train_loss: 0.10046356, train_accuracy: 0.9688, test_Accuracy: 0.9536
Epoch: [ 0] [  295/  468] time: 43.7795, train_loss: 0.26161665, train_accuracy: 0.9297, test_Accuracy: 0.9535
Epoch: [ 0] [  296/  468] time: 43.9216, train_loss: 0.09302892, train_accuracy: 0.9766, test_Accuracy: 0.9511
Epoch: [ 0] [  297/  468] time: 44.0592, train_loss: 0.24221586, train_accuracy: 0.9297, test_Accuracy: 0.9494
Epoch: [ 0] [  298/  468] time: 44.1986, train_loss: 0.17735702, train_accuracy: 0.9375, test_Accuracy: 0.9488
Epoch: [ 0] [  299/  468] time: 44.3368, train_loss: 0.29439765, train_accuracy: 0.9453, test_Accuracy: 0.9479
Epoch: [ 0] [  300/  468] time: 44.4751, train_loss: 0.13073365, train_accuracy: 0.9609, test_Accuracy: 0.9469
Epoch: [ 0] [  301/  468] time: 44.6131, train_loss: 0.13106191, train_accuracy: 0.9766, test_Accuracy: 0.9481
Epoch: [ 0] [  302/  468] time: 44.7537, train_loss: 0.23460494, train_accuracy: 0.9375, test_Accuracy: 0.9495
E

Epoch: [ 0] [  368/  468] time: 53.7784, train_loss: 0.29960328, train_accuracy: 0.9219, test_Accuracy: 0.9532
Epoch: [ 0] [  369/  468] time: 53.9129, train_loss: 0.09133118, train_accuracy: 0.9688, test_Accuracy: 0.9515
Epoch: [ 0] [  370/  468] time: 54.0485, train_loss: 0.14881161, train_accuracy: 0.9375, test_Accuracy: 0.9531
Epoch: [ 0] [  371/  468] time: 54.1831, train_loss: 0.22889756, train_accuracy: 0.9219, test_Accuracy: 0.9559
Epoch: [ 0] [  372/  468] time: 54.3200, train_loss: 0.08989510, train_accuracy: 0.9766, test_Accuracy: 0.9597
Epoch: [ 0] [  373/  468] time: 54.4549, train_loss: 0.12667422, train_accuracy: 0.9531, test_Accuracy: 0.9589
Epoch: [ 0] [  374/  468] time: 54.5908, train_loss: 0.17387651, train_accuracy: 0.9297, test_Accuracy: 0.9574
Epoch: [ 0] [  375/  468] time: 54.7258, train_loss: 0.18226364, train_accuracy: 0.9688, test_Accuracy: 0.9563
Epoch: [ 0] [  376/  468] time: 54.8625, train_loss: 0.20695949, train_accuracy: 0.9297, test_Accuracy: 0.9547
E

Epoch: [ 0] [  442/  468] time: 63.9910, train_loss: 0.11468755, train_accuracy: 0.9531, test_Accuracy: 0.9596
Epoch: [ 0] [  443/  468] time: 64.1427, train_loss: 0.15960231, train_accuracy: 0.9531, test_Accuracy: 0.9607
Epoch: [ 0] [  444/  468] time: 64.2944, train_loss: 0.10446216, train_accuracy: 0.9609, test_Accuracy: 0.9610
Epoch: [ 0] [  445/  468] time: 64.4442, train_loss: 0.09737605, train_accuracy: 0.9844, test_Accuracy: 0.9606
Epoch: [ 0] [  446/  468] time: 64.5937, train_loss: 0.09772941, train_accuracy: 0.9688, test_Accuracy: 0.9604
Epoch: [ 0] [  447/  468] time: 64.7428, train_loss: 0.08337691, train_accuracy: 0.9844, test_Accuracy: 0.9597
Epoch: [ 0] [  448/  468] time: 64.8910, train_loss: 0.08652875, train_accuracy: 0.9766, test_Accuracy: 0.9593
Epoch: [ 0] [  449/  468] time: 65.0392, train_loss: 0.05644247, train_accuracy: 0.9844, test_Accuracy: 0.9591
Epoch: [ 0] [  450/  468] time: 65.1886, train_loss: 0.16568241, train_accuracy: 0.9453, test_Accuracy: 0.9593
E